# Antenna Design Calculations
#### RF Characteristics
The operating frequency is defined by the WiFi Channel 802.11b standard.

$ Frequency\ (GHz): f = 2.412 $

$ Wavelength\ (m): \lambda = \frac{c}{f} $

In [1]:
import math
from tabulate import tabulate

# Constants
c = 299792458
f = 2.412*1e9

# Calculations
lamb = c/f

# Print Table
table = []
headers = ["Variable", "Symbol", "Value", "Unit"]
table.append(["Frequency", "f", f/1e9, "GHz"])
table.append(["Wavelength", "lamb", lamb, "m"])
print(tabulate(table, headers, floatfmt=".6f"))
print("\nNote: Frequency is stored in units of Hz but displayed in GHz for readability.")

Variable    Symbol       Value  Unit
----------  --------  --------  ------
Frequency   f         2.412000  GHz
Wavelength  lamb      0.124292  m

Note: Frequency is stored in units of Hz but displayed in GHz for readability.


#### Helical Antenna Parameters
The antenna's gain is a function of the number of turns in the helix.  The desired number will depend on experimental results, but a good starting point is $ n = 16 $.  The helix radius and pitch angle ($ 11 < r < 14 $ degrees optimal, according to Shiva) are based on MATLAB simulations that ensure the resultant parameters stay within the prescribed bounds.  

$ Number\ of\ Turns: n = 16 $

$ Radius\ (m): r = 0.019894 $

$ Pitch\ Angle\ (rad): \alpha = 0.226893 $

Basic design parameters are calculated using equations found in Chapter *X* of the ARRL Antenna Handbook.  It is recommended to keep the helix circumference within $ 0.75 \lambda < C_\lambda < 1.33 \lambda $ and the pitch between $ 0.2126 C_\lambda < S_\lambda < 0.2867 C_\lambda $.  These values will be checked in the code below.  

$ Circumference\ (m): C_\lambda = 2\pi r $

$ Pitch\ (m): S_\lambda = C_\lambda tan (\alpha) $

With these parameters defined, theoretical values for the gain are calculated.  A target gain of $ G \simeq 16 $ is desired.

$ Gain\ (dBi): G = 11.8 + 10 \log(2 n C_\lambda S_\lambda) $ *Note: Something must be wrong with this equation - it is spitting out terrible numbers.*

In [2]:
# Constants
n = 16
r = 0.019894
alpha = 0.226893
G_target = 16

# Calculations
C_lamb = 2*math.pi*r
C_lamb_lo = 0.75*lamb
C_lamb_hi = 1.33*lamb
S_lamb = math.tan(alpha)*C_lamb
G = 11.8 + 10*math.log(2*n*C_lamb*S_lamb)

# Print Table
table = []
table.append(["Turns", "n", n, "-"])
table.append(["Radius", "r", r, "m"])
table.append(["Pitch Angle", "alpha", alpha, "rad"])
table.append(["Circumference", "C_lamb", C_lamb, "m"])
table.append(["Pitch", "S_lamb", S_lamb, "m"])
table.append(["Gain", "G", G, "dBi"])
print(tabulate(table, headers, floatfmt=".6f"))

# Check Results
if C_lamb < C_lamb_lo or C_lamb > C_lamb_hi:
    print("\nWARNING: Circumference is out of spec!")
if S_lamb < 0.2126*C_lamb or S_lamb > 0.2867*C_lamb:
    print("\nWARNING: Pitch is out of spec!")
if G < G_target:
    print("\nWARNING: Gain is short by {:.3f} dBi!".format(G_target - G))

Variable       Symbol        Value  Unit
-------------  --------  ---------  ------
Turns          n         16.000000  -
Radius         r          0.019894  m
Pitch Angle    alpha      0.226893  rad
Circumference  C_lamb     0.124998  m
Pitch          S_lamb     0.028858  m
Gain           G         -9.790916  dBi



# Spring Design Calculations
#### Dimensional Parameters
Wire thickness has little effect on the RF characteristics of the antenna and will be chosen based on mechanical constraints.  
It is recommended to use wire diameters between $ 0.006 \lambda < d < 0.05 \lambda $.  Wire thicknesses below the minimum suggested value are possible but ought to be tested.  A range of reasonable wire gauges is presented in the table below.

\begin{array}{ l l }
\hline AWG & Diameter\ (mm) \\\hline
  18 & 1.024 \\\hline
  19 & 0.912 \\\hline
  20 & 0.812 \\\hline
  21 & 0.723 \\\hline
  22 & 0.644 \\\hline
  23 & 0.573 \\\hline
  24 & 0.511 \\\hline
\end{array}

This analysis will focus on 20 AWG but a range of values ought to be tested.

$ Wire\ Thickness\ (m): d = 0.000511 $

With that wire thickness defined, the rest of the spring's dimensional parameters are defined.

$ Mean\ Coil\ Diameter\ (m): D = 2r $

$ Outer\ Coil\ Diameter\ (m): D_o = D + d $

$ Inner\ Coil\ Diameter\ (m): D_i = D - d $

$ Pitch\ (m): p = S_\lambda $

The ratio of the spring's mean diameter to its wire thickness is defined as the spring index.  Suggested values are in the range of $ 4 < C_i < 12 $ yet the only cited downside for exceeding the maximum value is an increased tendency of the spring to become tangled.

$ Spring\ Index: C_i = \frac{D}{d} $



In [3]:
# Constants
d = 0.812/1e3 # 20 AWG
#d = 0.644/1e3 # 22 AWG
#d = 0.511/1e3 # 24 AWG

# Calculations
D = 2*r
D_o = D + d
D_i = D - d
p = S_lamb
C_i = D/d
C_i_lo = 4
C_i_hi = 12

# Print Table
table = []
table.append(["Wire Thickness", "d", d, "m"])
table.append(["Mean Diameter", "D", D, "m"])
table.append(["Outer Diameter", "D_o", D_o, "m"])
table.append(["Inner Diameter", "D_i", D_i, "m"])
table.append(["Pitch", "p", p, "m"])
table.append(["Spring Index", "C_i", C_i, "-"])
print(tabulate(table, headers, floatfmt=".6f"))

# Check Results
if C_i < C_i_lo or C_i > C_i_hi:
    print("\nWARNING: Spring index is out of spec!")

Variable        Symbol        Value  Unit
--------------  --------  ---------  ------
Wire Thickness  d          0.000812  m
Mean Diameter   D          0.039788  m
Outer Diameter  D_o        0.040600  m
Inner Diameter  D_i        0.038976  m
Pitch           p          0.028858  m
Spring Index    C_i       49.000000  -



#### Material Properties
Further analysis requires the specification of a material.  Beryllium copper has been chosen for its excellent electrical properties and common use as a spring material.  Actual tensile strengths of wire are highly depended on wire thickness and it is not wise to use generic material property values when dealing with such fine dimensions.  Table 10-5 in Chapter 10-6 of Shigley's  Mechanical Engineering Design lists values for beryllium copper wire.  Other sources list different values.  This will ultimately be updated with the supplier's numbers.  

$ Modulus\ of\ Elasticity\ (GPa): E \simeq 117.2 $

$ Shear\ Modulus\ of\ Elasticity\ (GPa): G \simeq 44.8 $

#### Static Loading Conditions
The stowed antenna must compress to less than 6 mm (or 17.6 mm if needed).  Unless it is wound and compressed flat (see *Wound Stowage* in the Helical Design notebook), the spring will be compressed to solid length prior to deployment.  Its "free length" will be the length to which it is fully deployed - more on this later.  

$ Free\ Length\ (m): L_0 = pn + d $

$ Solid\ Length\ (m): L_s = d(n + 1) $

The deflection of the spring is calculated as the difference between these lengths.

$ Deflection\ (m): y = L_0 - L_s $

With this defined, the force exterted by the fully compressed spring is calculated as function of the spring rate.

$ Spring\ Rate\ (N/m): k = \frac{d^4 G}{8D^3 n} $

$ Spring\ Force\ (N): F = ky $

In [4]:
# Constants
E = 117.2*1e9
G = 44.8*1e9
z_1 = 0.006
z_2 = 0.017

# Calculations
L_0 = p*n + d
L_s = d*(n+1)
y = L_0 - L_s
k = (d**4*G)/(8*D**3*n)
F = k*y

table = []
table.append(["Young's Modulus", "E", E/1e9, "GPa"])
table.append(["Shear Modulus", "G", G/1e9, "GPa"])
table.append(["Free Length", "L_0", L_0, "m"])
table.append(["Solid Length", "L_s", L_s, "m"])
table.append(["Deflection", "y", y, "m"])
table.append(["Spring Rate", "k", k, "N/m"])
table.append(["Spring Force", "F", F, "N"])
print(tabulate(table, headers, floatfmt=".6f"))
print("\nNote: Modulus values are stored in units of Pa but displayed in GPa for readability.")

if L_s > z_1:
    if L_s > z_2:
        print("\nWARNING: The compressed length is too large for the end card - coil must be wound!")
    else:
        print("\nThe compressed length is too large for the end cap - but will fit in the end card.")

Variable         Symbol         Value  Unit
---------------  --------  ----------  ------
Young's Modulus  E         117.200000  GPa
Shear Modulus    G          44.800000  GPa
Free Length      L_0         0.462540  m
Solid Length     L_s         0.013804  m
Deflection       y           0.448736  m
Spring Rate      k           2.415660  N/m
Spring Force     F           1.083994  N

Note: Modulus values are stored in units of Pa but displayed in GPa for readability.

The compressed length is too large for the end cap - but will fit in the end card.


####  Stress Analysis
The maximum shear stress of a spring can be calculated using any number of correction factors.  Shigley uses a single one: the curvature correction factor $ K_B $.

$ Curvature\ Correction\ Factor: K_B = \frac{4C_i+2}{4C_i-3} $ 

The maximum shear stress 

$ Maximum\ Shear\ Stress\ (GPa): \tau = K_B \frac{8FD}{\pi d^3} $


Material properties are dependent on a variety of factors including but not limited to:
- Wire Diameter
- Surface Treatment
- Alloy Composition
- Cold-Working
- Tempering
- Age Hardening
- Time Compressed (Stress Relaxation)

A wide range of values can be found from various sources.  Data will ultimately be sourced from the material supplier.  In the mean time, approximate values for cold-worked beryllium copper wire (source: NGK) will suffice.

$ Yield\ Strength\ (GPa): S_{y} \simeq 1.33 $

$ Tensile\ Strength\ (GPa): S_{ut} \simeq 1.41 $

Table 10-5 of Shigley tabulates, among other material properties, the torsional yield strength of beryllium copper as a function of its ultimate tensilte strength. 

$ Yield\ Strength\ in\ Torsion\ (GPa): S_{sy} = 0.5S_T $

If the maximum shear stress does not exceed the yield strength in torsion then the spring will not plastically deform when compressed between its free length and solid length.  The ratio of these values provides a factor of safety with a value of $ n_s > 1.2 $ being acceptable.

$ Factor\ of\ Safety\ (Solid\ Height): n_s = \frac{S_{sy}}{\tau} $

*(Am I missing anything?  See Shigley, Chapter 3.8 for details on the shear modulus of elasticity.)*

In [5]:
# Constants
S_y = 1.33*1e9
S_ut = 1.41*1e9

# Calculations
K_B = (4*C_i + 2)/(4*C_i - 3)
tau = K_B*((8*F*D)/(math.pi*d**3))
S_sy = 0.5*S_ut
n_s = S_sy/tau

# Print Table
table = []
table.append(["Curvature Correction Factor", "K_B", K_B, "-"])
table.append(["Largest Shear Stress", "tau", tau/1e9, "GPa"])
table.append(["Yield Strength", "S_y", S_y/1e9, "GPa"])
table.append(["Tensile Strength", "S_ut", S_ut/1e9, "GPa"])
table.append(["Yield Strength in Torsion", "S_sy", S_sy/1e9, "GPa"])
table.append(["Factor of Safety (Solid Height)", "n_s", n_s, "-"])
print(tabulate(table, headers, floatfmt=".6f"))
print("\nNote: Stress values are stored in units of Pa but displayed in GPa for readability.")

Variable                         Symbol       Value  Unit
-------------------------------  --------  --------  ------
Curvature Correction Factor      K_B       1.025907  -
Largest Shear Stress             tau       0.210455  GPa
Yield Strength                   S_y       1.330000  GPa
Tensile Strength                 S_ut      1.410000  GPa
Yield Strength in Torsion        S_sy      0.705000  GPa
Factor of Safety (Solid Height)  n_s       3.349887  -

Note: Stress values are stored in units of Pa but displayed in GPa for readability.


#### Free Length
Previous analysis assumed that the spring will be fully deployed at its free length.  However, the spring will be compressed by support tethers (see *Tethers* in the Helical Design Notebook).  The additional length is calculated as a function of the desired force that is split between the four tethers.

$ Tension\ (N): F_t =\ ??? $ *This value needs to be determined*

$ Total\ Length\ (M): y_t = y + \frac{4F_t}{k} $

In [7]:
# Constants
F_t = 1

# Calculations
y_t = y + (4*F_t)/k

# Print Table
table = []
table.append(["Tension", "F_t", F_t, "N"])
table.append(["Total Length", "y_t", y_t, "M"])
print(tabulate(table, headers, floatfmt=".6f"))

Variable      Symbol       Value  Unit
------------  --------  --------  ------
Tension       F_t       1.000000  N
Total Length  y_t       2.104598  M


# Conclusions
Theoretical analysis of 24 AWG beryllium copper wire indicates that a spring of the dimensions prescribed by the antenna design can be fully compressed to its solid height without yielding.  Such a compressed spring would fit well within the allotted space if it is mounted on the end card.  

# Next Steps
#### Tether Analysis
I'm assuming that the antenna will be slightly compressed by four tethers arranged around its circumferenced.  Let's do some static force analysis and figure out how much tension we need in order to keep the thing relatively rigid.  Alternatively, let's work backwards and just make the spring as long as we can on the lathe and see what kind of force that yields.  Should we limit ourselves to three tethers in order to increase the tension in each?  How does pinning the tether to each turn affect static analysis?  What do other teams do???
#### Manufacturing
- making on a lathe
- manually measured spring back rates
- add graphs and curve fit equations (generate in python?)
- both diameter and pitch

#### Wound Stowage Stress Analysis
Let's incorporate the bending stresses caused by winding the spring into a spiral (see *Wound Stowage* in the Helical Design notebook).  If those stresses seem reasonable then we will need to look at compressing to a plane rather than the spring's solid height.

#### Post-Manufacturing Heat Treatment
"Winding of the spring induces residual stresses through bending, but these are normal to the direction of the torsional working stresses in a coil spring.  Quite frequently in spring manufacture, they are relieved, after winding, by a mild thermal treatment."

#### Stress Relaxation
Apparently beryllium copper is resistant to stress relaxation.  It looks like most alloys retain 95% of their stress after being compressed for 1000 hours at room temperature.  Shiva will talk to professors for more insight into the problem before we dive into calculating or testing anything.

#### Refine Values
The following values should be updated with specifications from the material supplier:
- Modulus of Elasticity
- Shear Modulus of Elasticity
- Yield Strength
- Yield Strength in Torsion
- Tensile Strength